In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic-1/train.csv
/kaggle/input/titanic-1/test.csv
/kaggle/input/titanic-1/gender_submission.csv


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [34]:
train = pd.read_csv("/kaggle/input/titanic-1/train.csv")
test = pd.read_csv("/kaggle/input/titanic-1/test.csv")

In [35]:
print(train)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [36]:
train["Age"].fillna(train['Age'].median(), inplace = True)
test["Age"].fillna(test['Age'].median(), inplace = True)
test["Fare"].fillna(test['Fare'].median(),inplace = True)
train["Fare"].fillna(train['Fare'].median(),inplace = True)
test["Embarked"].fillna(test['Embarked'].mode()[0],inplace = True)
train["Embarked"].fillna(train['Embarked'].mode()[0],inplace = True)

/tmp/ipykernel_36/3630715357.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train["Age"].fillna(train['Age'].median(), inplace = True)
/tmp/ipykernel_36/3630715357.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

In [26]:
# train["Sex"] = train["Sex"].map({'male':0,'female': 1})
# test["Sex"] = test["Sex"].map({'male':0,'female': 1})
# test["Embarked"] = test["Embarked"].map({'S':0, 'C':1, 'Q':2})
# train["Embarked"] = train["Embarked"].map({'S':0, 'C':1, 'Q':2})

In [37]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train["Sex"] = encoder.fit_transform(train["Sex"])
test["Sex"] = encoder.fit_transform(test["Sex"])
test["Embarked"] = encoder.fit_transform(test["Embarked"])
train["Embarked"] = encoder.fit_transform(train["Embarked"])


In [38]:
features = ['Pclass' , 'Sex' , 'Age' , 'Fare', 'SibSp' , 'Parch', 'Embarked']
X = train[features]
y = train['Survived']

In [39]:
print(X)
print(y)

     Pclass  Sex   Age     Fare  SibSp  Parch  Embarked
0         3    1  22.0   7.2500      1      0         2
1         1    0  38.0  71.2833      1      0         0
2         3    0  26.0   7.9250      0      0         2
3         1    0  35.0  53.1000      1      0         2
4         3    1  35.0   8.0500      0      0         2
..      ...  ...   ...      ...    ...    ...       ...
886       2    1  27.0  13.0000      0      0         2
887       1    0  19.0  30.0000      0      0         2
888       3    0  28.0  23.4500      1      2         2
889       1    1  26.0  30.0000      0      0         0
890       3    1  32.0   7.7500      0      0         1

[891 rows x 7 columns]
0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64


In [40]:
X_train, X_val, y_train, y_val = train_test_split(X , y, test_size = 0.2, random_state = 42)
model = LogisticRegression(max_iter = 200)

In [41]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=200)

In [42]:
print(model.score(X_val, y_val))

0.8100558659217877


In [45]:
final_preds =  model.predict(test[features])
submission = pd.DataFrame(
    {
        "PassengerId": test['PassengerId'],
        'Survived' : final_preds
    }
)
submission.to_csv('submission.csv', index = False)
print("!!!!!!")

!!!!!!
